In [4]:
import numpy as np
import scipy as sci

In [5]:
# teeth calcs

#parameters
target_ratio_1 = 1.75
target_ratio_2 = .9

#constraints
Ni1 = 16
# all teeth numbers in order Ni1, N01, Ni2, No2, must be integers

#teeth numbers found with this equation
teeth_ratio_matrix = np.array([[1, 1, -1, -1], [target_ratio_1, -1, 0, 0], [0, 0, target_ratio_2, -1], [1, 0, 0, 0]])
b = np.array([0, 0, 0, Ni1])
teeth = np.linalg.solve(teeth_ratio_matrix, b)

def loss(ratio1, ratio2, target_ratio1, target_ratio2):
    return np.linalg.norm(ratio1-target_ratio1) + np.linalg.norm(ratio2-target_ratio2)


In [6]:
# teeth numbers given ratios

#module
m = 1.5

# Target ratios
target_ratio_1 = 2.1
target_ratio_2 = 1.1

# Fixed value
Ni1 = 20

# Search range
search_range = range(10, 100)

best_loss = float('inf')
best_solution = None

for No1 in search_range:
    ratio1 = No1 / Ni1
    for Ni2 in search_range:
        for No2 in search_range:
            # Gear train constraint: Ni1 + No1 == Ni2 + No2
            if Ni1 + No1 != Ni2 + No2:
                continue

            ratio2 = No2 / Ni2
            loss = abs(ratio1 - target_ratio_1) + abs(ratio2 - target_ratio_2)
            if loss < best_loss:
                best_loss = loss
                best_solution = (Ni1, No1, Ni2, No2)

print(f"Best solution: Ni1={best_solution[0]}, No1={best_solution[1]}, Ni2={best_solution[2]}, No2={best_solution[3]}")
print(f"Ratios: {best_solution[1]/best_solution[0]:.4f}, {best_solution[3]/best_solution[2]:.4f}")
print(f"Loss: {best_loss:.6f}")
print(f"axle distance: {(best_solution[0]/2*m) + (best_solution[1]/2*m)}")

Best solution: Ni1=20, No1=42, Ni2=30, No2=32
Ratios: 2.1000, 1.0667
Loss: 0.033333
axle distance: 46.5


In [7]:
#look up values in table on page 738 shigley's
data = [
    [12, 0.245],
    [13, 0.261],
    [14, 0.277],
    [15, 0.290],
    [16, 0.296],
    [17, 0.303],
    [18, 0.309],
    [19, 0.314],
    [20, 0.322],
    [21, 0.328],
    [22, 0.331],
    [24, 0.337],
    [26, 0.346],
    [28, 0.353],
    [30, 0.359],
    [34, 0.371],
    [38, 0.384],
    [43, 0.397],
    [50, 0.409],
    [60, 0.422],
    [75, 0.435],
    [100, 0.447],
    [150, 0.460],
    [300, 0.472],
    [400, 0.480],
]

Y_values = np.array(data)
# print(Y_values)

In [8]:
# Lewis bending stress

m = 1.5
torque = 4.8 #Nm, actual motor torque is 4.8
n_teeth = 22
rpm = 5400
F = 15 #mm
Y = np.interp(n_teeth, Y_values[:,0], Y_values[:,1])

radius = m*n_teeth/2 #mm
Wt = torque / (radius/1000)
vel = (radius/1000) * rpm * 2 * np.pi / 60 #m/sec
Kv = (6.1 + vel) / 6.1

bending_stress = (Kv * Wt) / (F * m * Y)
print(f"lewis bending stress (MPa): {bending_stress}")

lewis bending stress (MPa): 98.80929362910945


In [9]:
# contact stress

m = 1.5
torque = 5 #Nm
n_teeth1 = 22
n_teeth2 = 46
rpm = 5400
F = 15 #mm
v_pois = .25 #poissons ratio
E = 193*(10**9) #youngs modulus, Pa
pressure_angle = np.deg2rad(20)

radius = m * n_teeth1 /2 #mm
Wt = torque / (radius/1000) #N
vel = (radius/1000) * rpm * 2 * np.pi / 60 #m/sec
Kv = (6.1 + vel) / 6.1

r1 = n_teeth1 * m * np.sin(pressure_angle) / 2
r2 = n_teeth2 * m * np.sin(pressure_angle) / 2

Cp = np.sqrt(1 / (np.pi*((1-v_pois**2)*2)/E))

contact_stress = Cp * np.sqrt((Kv * Wt) / (F/1000 * np.cos(pressure_angle)) * ((1/r1) + (1/r2)))
print(f"contact stress (MPa): {contact_stress/(10**6)}")

contact stress (MPa): 21.604330238852693


$100kg * 9.8\frac{m}{s^2} * 1(\mu) * 1.847\frac{wheel_rad.}{rotor_rad.} = 1456N$

$-1456sin(45) + A_x = 0$